
<h1 style="font-family:verdana;font-size:300%;text-align:center;background-color:#f2f2f2;color:#0d0d0d">AMMI_2024_NLP - Week 1</h1>

<h1 style="font-family:verdana;font-size:180%;text-align:Center;color:#993333"> Lab 3: N-gram models </h1>


Before we start, please change the name of the notebook to the following format : **Firstname_LASTNAME_Lab3__N_gram_models.ipynb**


In some cells and files you will see code blocks that look like this:

```python
##############################################################################
#                    TODO: Write the equation for a line                     #
##############################################################################
pass
##############################################################################
#                              END OF YOUR CODE                              #
##############################################################################
```

You should replace the `pass` statement with your own code and leave the blocks intact, like this:

```python
##############################################################################
#                    TODO: Write the equation for a line                     #
##############################################################################
y = m * x + b
##############################################################################
#                              END OF YOUR CODE                              #
##############################################################################
```

In [4]:
import io, sys, math, re
from collections import defaultdict
import numpy as np

In [5]:
# data_loader
def load_data(filename):
    '''
    parameters:
    filename (string): datafile

    Returns:
    data (list of lists): each list is a sentence of the text
    vocab (dictionary): {word: no of times it appears in the text}
    '''
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    vocab = defaultdict(lambda:0)
    for line in fin:
        sentence = line.split()
        data.append(sentence)
        for word in sentence:
            vocab[word] += 1
    return data, vocab

In [6]:

print("load training set..")
print("\n")
train_data, vocab = load_data("/content/drive/MyDrive/AMMI-23/nlp/Lab3/train1.txt")
print(train_data[0])
print("\n")
print("how :",vocab['how'])
print("load validation set")
valid_data, _ = load_data("/content/drive/MyDrive/AMMI-23/nlp/Lab3/valid1.txt")


load training set..


['<s>', 'my', 'fathers', "don't", 'speak', 'dutch.', '</s>']


how : 107
load validation set


In [7]:
def remove_rare_words(data, vocab, mincount = 1):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text
    vocab (dictionary): {word: no of times it appears in the text}
    mincount(int): the minimum count

    Returns:
    data_with_unk(list of lists): data after replacing rare words with <unk> token
    '''
    # replace words in data that are not in the vocab
    # or have a count that is below mincount
    data_with_unk = []
    ##########################################################################
    #                      TODO: Implement this function                     #
    ##########################################################################
    # Replace "pass" statement with your code
    for sentence in data:
      new_sentence = []
      for word in sentence:
        if vocab[word] < mincount:
          new_sentence.append('<unk>')
        else:
          new_sentence.append(word)
      data_with_unk.append(new_sentence)
    ##########################################################################
    #                            END OF YOUR CODE                            #
    ##########################################################################

    return data_with_unk

In [8]:
print("remove rare words")

train_data = remove_rare_words(train_data, vocab, mincount = 2)
valid_data = remove_rare_words(valid_data, vocab, mincount = 1)
#train_data
print(train_data[0])

remove rare words
['<s>', 'my', '<unk>', "don't", 'speak', '<unk>', '</s>']


In [9]:
def build_ngram(data, n):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text
    n (int): size of the n-gram

    Returns:
    prob (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    '''
    total_number_words = 0
    counts = defaultdict(lambda: defaultdict(lambda: 0.0))

    def _build_ngram(data,n):
      counts = defaultdict(lambda: defaultdict(lambda: 0.0))
      for sentence in data:
          sentence = tuple(sentence)
          ##########################################################################
          #                      TODO: Implement this function                     #
          # dict can be indexed by tuples
          # store in the same dict all the ngrams
          # by using the context as a key and the word as a value
          ##########################################################################
          # Replace "pass" statement with your code
          for i in range(len(sentence) - n + 1):
              context = tuple(sentence[i:i+n-1])
              word = sentence[i+n-1]
              counts[context][word] += 1
          ##########################################################################
          #                            END OF YOUR CODE                            #
          ##########################################################################

      prob = defaultdict(lambda: defaultdict(lambda: 0.0))
      # Build the probabilities from the counts
      # Be careful with how you normalize!

      for context in counts.keys():
        # p(w | context) = count(context, w)/ count(context)
        ##########################################################################
        #                      TODO: Implement this function                     #
        ##########################################################################
        # Replace "pass" statement with your code
          total_count = float(sum(counts[context].values()))
          for word in counts[context]:
              prob[context][word] = counts[context][word] / total_count
        ##########################################################################
        #                            END OF YOUR CODE                            #
        ##########################################################################

      return prob

    prob = defaultdict(lambda: defaultdict(lambda: 0.0)) # Replace "..." statement with your code
    for i in range(1,n+1):
      ngram_prob = _build_ngram(data, i)
      for context in ngram_prob:
        for word in ngram_prob[context]:
          prob[context][word] = ngram_prob[context][word] # Replace "..." statement with your code

    return prob

In [24]:
# RUN TO BUILD NGRAM MODEL
n = 10
print("build ngram model with n = ", n)
model = build_ngram(train_data, n)

build ngram model with n =  10


Here, implement a recursive function over shorter and shorter context to compute a "stupid backoff model". An interpolation model can also be implemented this way.

In [11]:
def get_prob(model, context, w, alpha=0.4):
    '''
    Parameters:
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    context (list of strings): a sentence
    w(string): the word we need to find it's probability given the context

    Retunrs:
    prob(float): probability of this word given the context
    '''

    # code a recursive function over
    # smaller and smaller context
    # to compute the backoff model
    ##########################################################################
    #                      TODO: Implement this function                     #
    ##########################################################################
    # Replace "pass" statement with your code
    context_tuple = tuple(context)
    if context_tuple in model and w in model[context_tuple]:
      return model[context_tuple][w]

    if len(context) == 0:
      return model[tuple()]['<unk>'] if '<unk>' in model[tuple()] else 0.0
    ##########################################################################
    #                            END OF YOUR CODE                            #
    ##########################################################################

    return alpha * get_prob(model, context[1:], w, alpha)

In [12]:
def perplexity(model, data, n):
    '''
    Parameters:
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    data (list of lists): each list is a sentence of the text
    n(int): size of the n-gram

    Retunrs:
    perp(float): the perplexity of the model
    '''
    ##########################################################################
    #                      TODO: Implement this function                     #
    ##########################################################################
    # Replace "pass" statement with your code
    total_log_prob = 0.0
    total_words = 0

    for sentence in data:
      sentence = ['<s>'] * (n - 1) + sentence + ['</s>']
      for i in range(n - 1, len(sentence)):
        context = sentence[i - n + 1:i]
        word = sentence[i]
        prob = get_prob(model, context, word)
        total_log_prob += np.log(prob) if prob > 0 else -np.inf
        total_words += 1

    avg_log_prob = total_log_prob / total_words
    perp = np.exp(-avg_log_prob)
    ##########################################################################
    #                            END OF YOUR CODE                            #
    ##########################################################################

    return perp

In [66]:
# COMPUTE PERPLEXITY ON VALIDATION SET
model = build_ngram(train_data, n)
print("The perplexity is", perplexity(model, valid_data, n=n))

The perplexity is 3975.5446718614767


In [15]:
def get_proba_distrib(model, context):
    ## need to get the the words after the context and their probability of appearance
    ## after this context
    '''
    Parameters:
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    context (list of strings): the sentence we need to find the words after it and
    thier probabilites

    Retunrs:
    words_and_probs(dic): {word: probability of word given context}

    '''
    # code a recursive function over context
    # to find the longest available ngram
    ##########################################################################
    #                      TODO: Implement this function                     #
    ##########################################################################
    # Replace "pass" statement with your code
    context_tuple = tuple(context)

    if context_tuple in model:
      return model[context_tuple]

    if len(context) == 0:
      return model[tuple()]
    ##########################################################################
    #                            END OF YOUR CODE                            #
    ##########################################################################

    return get_proba_distrib(model, context[1:])

In [16]:
def generate(model):
    '''
    Parameters:
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }

    Retunrs:
    sentence (list of strings): a sentence sampled according to the language model.


    '''
    # generate a sentence. A sentence starts with a <s> and ends with a </s>
    # Possiblly a use function is:
    # np.random.choice(x, 1, p = y)

    # where x is a list of things to sample from
    # and y is a list of probability (of the same length as x)
    sentence = ["<s>"]
    n =10
    #print (model[("<s>")])
    #print (len(model[tuple(sentence)].values()))
    while sentence[-1] != "</s>" and len(sentence)<10:
        ##########################################################################
        #                      TODO: Implement this function                     #
        ##########################################################################
        # Replace "pass" statement with your code
        context = sentence[-(n-1):]
        context_probs = get_proba_distrib(model, context)

        if not context_probs:
            break

        words = list(context_probs.keys())
        probs = list(context_probs.values())

        next_word = np.random.choice(words, p=probs)
        sentence.append(next_word)

    if sentence[-1] != "</s>":
        sentence.append("</s>")
        ##########################################################################
        #                            END OF YOUR CODE                            #
        ##########################################################################

    return sentence

In [17]:
# GENERATE A SENTENCE FROM THE MODEL
model = build_ngram(train_data, n)
print("Generated sentence: ",generate(model))

Generated sentence:  ['<s>', 'tom', 'and', 'mary', 'said', 'they', 'intend', 'to', 'do', 'that.', '</s>']


Once you are done implementing the model, evaluation and generation code, you can try changing the value of `n`, and play with a larger training set (`train2.txt` and `valid2.txt`). You can also try to implement an interpolation model.

In [18]:
print("load training set 2..")
print("\n")
train_data2, vocab = load_data("/content/drive/MyDrive/AMMI-23/nlp/Lab3/train2.txt")
print(train_data2[0])
print("\n")
print("how :",vocab['how'])
print("load validation set 2")
valid_data2, _ = load_data("/content/drive/MyDrive/AMMI-23/nlp/Lab3/valid2.txt")


load training set 2..


['<s>', 'i', 'liked', 'your', 'idea', 'and', 'adopted', 'it', '.', '</s>']


how : 3195
load validation set 2


In [19]:
n = 3

In [20]:
model = build_ngram(train_data2, n)

In [21]:
perplexity(model,valid_data2, n)

43.51208100017771

In [23]:
generate(model)

['<s>', 'when', 'you', 'write', 'to', 'me', '.', '</s>']